In [ ]:
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain_core.documents import Document
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
import os

# 1. Hugging Face 다운로드 없이 실행 가능한 모델 설정
MODEL_NAME = "facebook/opt-1.3b"  # 더 가벼운 모델 사용 (1.3B)

# 2. 4bit 양자화 설정 (RAM 절약)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="float16",  # 4bit 연산을 float16으로 설정하여 성능 최적화
    bnb_4bit_use_double_quant=True  # 더블 양자화 적용 (메모리 절약)
)

# 3. Hugging Face 토크나이저 및 모델 로드 (다운로드 없이 실행)
print("모델 로드 중... (facebook/opt-1.3b)")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME
    # 최신 방식으로 4bit 적용
)
print("모델 로드 완료!")

# 4. LangChain용 LLM 파이프라인 변환
llm_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, max_length=512)
llm = HuggingFacePipeline(pipeline=llm_pipeline)

# 5. 문서 데이터 추가 (더 많은 정보 포함)
docs = [
    "LangChain은 AI 개발을 쉽게 하는 프레임워크입니다. Python 기반이며, LLM을 연결하여 자연어 처리 AI를 만들 수 있습니다.",
    "RAG는 검색을 활용하여 AI 답변을 향상시키는 기술입니다. 벡터 검색을 통해 더 정확한 정보를 제공합니다.",
    "LangChain은 다양한 데이터 소스를 활용할 수 있습니다. SQL, CSV, JSON 같은 외부 데이터도 지원합니다.",
    "RAG는 GPT 모델과 결합하면 더욱 강력한 성능을 발휘합니다. OpenAI GPT-4와 함께 사용하면 정확도가 높아집니다.",
    "LangChain은 기업용 AI 개발에도 활용됩니다. 챗봇, 데이터 분석, 검색 엔진 등의 분야에서 사용됩니다.",
    "RAG는 기존 검색 엔진과 다르게, 벡터 검색을 활용하여 문맥을 더 잘 이해합니다.",
    "LangChain은 OpenAI, Hugging Face, Cohere 같은 다양한 모델과 연결할 수 있습니다.",
    "LangChain을 사용하면 다양한 데이터베이스를 검색하여 AI가 더 좋은 답변을 제공할 수 있습니다."
]
documents = [Document(page_content=text) for text in docs]

# 6. Hugging Face Embeddings 생성 (벡터 변환)
embeddings = HuggingFaceEmbeddings()

# 7. FAISS 벡터스토어 저장 및 로드 기능 추가
faiss_index_path = "faiss_index"

if os.path.exists(faiss_index_path):
    print("기존 FAISS 벡터스토어 로드 중...")
    vectorstore = FAISS.load_local(faiss_index_path, embeddings)
else:
    print("새로운 FAISS 벡터스토어 생성 중...")
    vectorstore = FAISS.from_documents(documents, embeddings)
    vectorstore.save_local(faiss_index_path)  # 벡터 저장 (재사용 가능)

# 8. RetrievalQA 체인 설정 (검색 + 생성 결합)
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(),
    return_source_documents=True  # 🔹 검색된 문서를 함께 반환
)

# 9. 사용자 질문을 입력하고 검색된 문서를 활용하여 답변 생성
query = "RAG란?"
result = qa_chain({"query": query})

# 10. 결과 출력
print("\n🔹 AI의 답변:")
print(result["result"])

# 검색된 문서 출력
print("\n🔹 검색된 문서:")
for doc in result["source_documents"]:
    print(f"- {doc.page_content}")




✅ 모델 로드 중... (facebook/opt-1.3b)


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


✅ 모델 로드 완료!


/tmp/ipykernel_47487/3827639429.py:46: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
/tmp/ipykernel_47487/3827639429.py:68: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  result = qa_chain({"query": query})
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


✅ 새로운 FAISS 벡터스토어 생성 중...

🔹 AI의 답변:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

RAG는 기존 검색 엔진과 다르게, 벡터 검색을 활용하여 문맥을 더 잘 이해합니다.

RAG는 검색을 활용하여 AI 답변을 향상시키는 기술입니다. 벡터 검색을 통해 더 정확한 정보를 제공합니다.

LangChain은 기업용 AI 개발에도 활용됩니다. 챗봇, 데이터 분석, 검색 엔진 등의 분야에서 사용됩니다.

LangChain을 사용하면 다양한 데이터베이스를 검색하여 AI가 더 좋은 답변을 제공할 수 있습니다.

Question: RAG란?
Helpful Answer:

저는 정보를 제공하여 정보�

🔹 검색된 문서:
- RAG는 기존 검색 엔진과 다르게, 벡터 검색을 활용하여 문맥을 더 잘 이해합니다.
- RAG는 검색을 활용하여 AI 답변을 향상시키는 기술입니다. 벡터 검색을 통해 더 정확한 정보를 제공합니다.
- LangChain은 기업용 AI 개발에도 활용됩니다. 챗봇, 데이터 분석, 검색 엔진 등의 분야에서 사용됩니다.
- LangChain을 사용하면 다양한 데이터베이스를 검색하여 AI가 더 좋은 답변을 제공할 수 있습니다.
